# Создание Телеграм-ботов. Боты в группах и каналах 2.

Сегодня мы с вами продолжим использовать Телеграм-ботов в группах и каналах. Научимся отслеживать участников в чате, делать языковую локализацию и отправлять эмодзи через бота.

---

Ссылки:
* Описание всех методов и классов Telegram Bot API: https://core.telegram.org/bots/api
* Гайд по созданию Телеграм-ботов: https://mastergroosha.github.io/telegram-tutorial/

# Как понять, какие пользователи есть в группе. 

К сожалению, нет методов, которые бы возвращали список участников группы. Доступны методы:  

* `bot.get_chat_member(chat_id, user_id)` - если такой пользователь есть в чате - возвращает объект `ChatMember`, в котором содержатся объект `User` и разрешения пользователей.
* `bot.get_chat_administrators(chat_id)` - возращает список объектов `ChatMember` с администраторами группы.
* `bot.get_chat_members_count(chat_id)` - возвращает количество участников группы.

Если нужны все пользователи - можно отслеживать добавление и удаление участников.

In [ ]:
# Добавление участников
@bot.message_handler(content_types=['new_chat_members'])  
def new_member(message):
    bot.delete_message(message.chat.id, message.message_id)  # Удаляем системное сообщение о добавлении
    
    for user in message.new_chat_members: # Список новых участников
        bot.send_message(message.chat.id, "Обнаружен новый участник - {}. Его id - {}".format(user.username, user.id))
        

# Удаление участника
@bot.message_handler(content_types=['left_chat_member'])
def delete_member(message):
    bot.delete_message(message.chat.id, message.message_id)  # Удаляем системное сообщение об удалении
    
    user = message.left_chat_member
    bot.send_message(message.chat.id, "Нас покинул участник - {}".format(user.username))

# Языковая локализация

Наш бот мог бы присылать сообщения на разных языках в зависимости от языка пользователя. Как это реализовать?  

В объекте `User` (например, в поле `message.from_user`) есть поле `language_code`, содержащее языковую метку пользователя.  

Языковые метки могут быть `ru`, `en-GB`, `en-US` и так далее. Иногда `language_code` может быть `None`. Как видите, даже для одного языка могут быть разные метки в зависимости от региона.  
Подробнее, здесь: https://en.wikipedia.org/wiki/IETF_language_tag

Если нужно разделение на русский и английский - достаточно такого кода:

In [ ]:
def get_language(lang_code):
    if not lang_code:
        return 'en'
    
    lang_code = lang_code.split('-')[0]  # Считаем, что везде сначала идет язык, потом дефис, потом регион.
    
    if lang_code == 'ru':
        return 'ru'
    else:
        return 'en'

# Эмодзи

Чтобы отправить эмодзи вы на его месте должны указать его код `UTF-8`.  
Список кодов, соответствующих разным смайликам - здесь: https://apps.timwhitlock.info/emoji/tables/unicode


In [ ]:
# Если вы отправляете только смайлик - можно просто указать параметром его байтовое представление в UTF-8
bot.send_message(message.chat.id, b'\xF0\x9F\x98\x81')


# Если смайлик - часть теста - придется к байтовой записи применить команду decode('UTF-8')
bot.send_message(message.chat.id, 'Привет' + b'\xF0\x9F\x98\x81'.decode('UTF-8'))


# Если есть набор смайликов - можно задать их в байтовом виде в списке, а потом все раскодировать генератором.
smiles = [b'\xF0\x9F\x98\x81', b'\xF0\x9F\x98\x82', b'\xF0\x9F\x98\x85']
smiles = [value.decode('UTF-8') for value in smiles]
bot.send_message(message.chat.id, 'Привет {}'.format(smiles[2]))


# Чтобы узнать UTF-8 код смайлика - можно воспользоваться командой encode('UTF-8').
@bot.message_handler(func=lambda message: True)
def get_smiley(message):
    bot.send_message(message.chat.id, str(message.text.encode('UTF-8')))  # Привели к str, чтобы отобразидся код, а не смайлик

# Опросы
## Отправка


Боты умеют отправлять опросы. Для этого используется метод `bot.send_poll()`. Расскажу про аргументы этого метода:  

* `chat_id` - id чата
* `question` - вопрос (1-255 символов)
* `options` - список из вариантов ответа (2-10 вариантов, до 100 символов)
* `is_anonymous` - анонимный ли опрос
* `type` - тип (`quiz` - викторина или `regular` - опрос)
* `allows_multiple_answers` - доступен ли множественный ответ
* `correct_option_id` - номер верной опции, начиная с нуля (только для викторин)
* `explanation` - пояснение (появляется, если нажать на неправильный ответ или лампочку в викторине)
* `explanation_parse_mode` - как парсить пояснение (например, `'HTML'`)
* `open_period` - сколько по времени в секундах доступен опрос (5-600 секунд)
* `close_date` - в какое unix-время закрыть опрос (не меньше, чем через 5 секунд, не больше, чем через 600)
* `is_closed` - установить в `True`, если нужно закрыть опрос сразу же.
* `disable_notification` - отключить уведомления
* `reply_to_message_id` - на какое сообщение отвечать
* `reply_markup` - клавиатура для ответа


In [ ]:
bot.send_poll(message.chat.id, "Сколько будет два плюс два?", options=['1', '4', '2', '3'],
                  is_anonymous=False, type='quiz', correct_option_id=1, explanation='Учи математику, сынок')

## Получение

Отловить опрос можно этим хэндлером: `@bot.message_handler(content_types=['poll'])`.  
В `message` вместо поля `text` нам придет объект `Poll` - опрос. Разберемся, как у него есть поля:  

* `id` - id опроса
* `question` - вопрос
* `options` - список объектов `PollOption` (есть поля `text` и `voter_count` - количество проголосовавших за этот вариант)
* `total_voter_count` - количество проголосовавших в опросе
* `is_closed`
* `is_anonymous`
* `type`
* `allows_multiple_answers`
* `correct_option_id`
* `explanation`
* `explanation_entities` - список entities в объяснении
* `open_period`
* `close_date`

Вот так, например, можно отправить пользователю его опрос в ответ:

In [ ]:
@bot.message_handler(content_types=['poll'])
def echo(message):
    p = message.poll
    options = [option.text for option in p.options]  # Сами создаем список с опциями, т.к. нам нужно взять только их текст

    bot.send_poll(message.chat.id, p.question, options, is_anonymous=p.is_anonymous,
                  type=p.type, allows_multiple_answers=p.allows_multiple_answers,
                  correct_option_id=p.correct_option_id, explanation=p.explanation)

# Получение результатов опроса

Каждый раз, когда пользователь выбирает ответ в неанонимном голосовании - это может отловить `@bot.poll_answer_handler(func=lambda answer: True)`.  

В него приходит объект `PollAnswer` с полями:
* `poll_id` - id опроса
* `user` - объект `User` с информацией о голосовавшем
* `options_ids` - список с номерами выбранных опций

# Дописываем бота-администратора группы

In [ ]:
# Код в проекте PyCharm из предыдущей лекции.